In [1]:
import os
import numpy as np
import pandas as pd
import itertools as it
from glob import glob
import matplotlib.pyplot as plt
from sbpy.calib import Sun
from astropy import units as u
from astropy.io import fits

from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from astropy.visualization import ImageNormalize, PercentileInterval, AsinhStretch

from lvmdrp.core.constants import LVM_UNAM_URL
from lvmdrp.utils.examples import fetch_example_data, get_frames_metadata, fix_lamps_metadata, get_masters_metadata

from lvmdrp.core import rss, image, spectrum1d
from lvmdrp.functions import imageMethod, rssMethod, fluxCalMethod

plt.style.use("seaborn-v0_8-talk")

In [2]:
# define input data directory
data_path = os.path.abspath(os.path.join("..", "data"))

# let's create the output directory
output_path = "./data"
os.makedirs(output_path, exist_ok=True)

# create processed frames path template
out_main_path = os.path.join(output_path, "lvm-{kind}-{camera}-{expnum}.fits")
out_calib_path = os.path.join(output_path, "lvm-{mjd}-{kind}-{camera}-{exptime}.fits")

# overwrite or not
OVERWRITE = True

In [3]:
# extract metadata
frames_table = get_frames_metadata(path=os.path.join(data_path, "lco_com"), ignore_cache=False).to_pandas()
frames_table = frames_table.loc[~frames_table.mjd.isin([60008,60009,60037,60038])]
# fix arc lamps metadata
# fix_lamps_metadata(metadata=frames_table, lamp_names="argon neon ldls hgne xenon krypton".split())

# get master calibration metadata
masters_bias = get_masters_metadata(path_pattern=out_calib_path, mjd="super", kind="mbias")
masters_dark = get_masters_metadata(path_pattern=out_calib_path, mjd="super", kind="mdark").sort_values("exptime")

[INFO]: loading cached metadata from '/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/frames_table.pkl'


In [4]:
# define list of fiberflat exposures
expnum_list = list(range(193,200)) + list(range(232,237)) + list(range(559,594)) + list(range(694, 697))

q = "mjd>=60039 & (imagetyp=='flat' | ldls)"
fiberflat_analogs = frames_table.query(q).loc[(frames_table.expnum.isin([f"{i:08d}" for i in expnum_list]))].groupby(["mjd", "camera", "exptime"])

for mjd, camera, exptime in fiberflat_analogs.groups:
    fiberflats = fiberflat_analogs.get_group((mjd, camera, exptime))
    print(fiberflats.to_string())
    
    master_bias = masters_bias.query("camera==@camera")
    master_dark = masters_dark.query("camera==@camera").sort_values("exptime")
    try:
        mbias = master_bias.iloc[0]
    except IndexError:
        imageMethod.log.error(f"no master bias found for {mjd = }, {camera = }, {exptime = }")
    try:
        mdark = master_dark.iloc[0]
    except IndexError:
        imageMethod.log.error(f"no master dark found for {mjd = }, {camera = }, {exptime = }")
        continue
    
    # let's pre-process the all fiberflat frames
    cfiberflat_paths = []
    for _, fiberflat_frame in fiberflats.iterrows():
        fiberflat_path = fiberflat_frame["path"]
        fiberflat_expnum = fiberflat_frame["expnum"]

        pfiberflat_path = out_main_path.format(kind="pfiberflat", camera=camera, expnum=fiberflat_expnum)

        cfiberflat_path = out_main_path.format(kind="cfiberflat", camera=camera, expnum=fiberflat_expnum)
        cfiberflat_paths.append(cfiberflat_path)
        if not os.path.exists(cfiberflat_path) or OVERWRITE:
            imageMethod.preprocRawFrame_drp(
                in_image=fiberflat_path,
                out_image=pfiberflat_path,
                in_mask=out_calib_path.format(mjd="super", kind="mask", camera=camera, exptime="x"),
                subtract_overscan=True, assume_imagetyp="fiberflat"
            )
            imageMethod.detrendFrame_drp(
                in_image=out_main_path.format(kind="pfiberflat", camera=camera, expnum=fiberflat_expnum),
                out_image=cfiberflat_path,
                in_bias=mbias.path,
                in_dark=mdark.path,
                reject_cr=1,
            )

    imageMethod.createMasterFrame_drp(
        in_images=cfiberflat_paths,
        out_image=out_calib_path.format(mjd=mjd, kind="mfiberflat", camera=camera, exptime=int(fiberflats.exptime.min())),
    )

    imagetyp spec    mjd camera    expnum  exptime  argon   neon  ldls   hgne  xenon  krypton                                                                                                                      path
340   object  sp3  60039     b3  00000559     10.0  False  False  True  False  False    False  /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/sas/sdsswork/data/lvm/lco/60039/sdR-s-b3-00000559.fits.gz
372   object  sp3  60039     b3  00000560     10.0  False  False  True  False  False    False  /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/sas/sdsswork/data/lvm/lco/60039/sdR-s-b3-00000560.fits.gz
[INFO]: starting preprocessing of raw image 'sdR-s-b3-00000559.fits.gz'
[WARNING]: assuming IMAGETYP = 'fiberflat'
[INFO]: exposure time 10.0 (s)
[WARNING]: assuming TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[WARNING]: assuming BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', 

In [5]:
fiberflat_analogs = frames_table.query(q).loc[(frames_table.expnum.isin([f"{i:08d}" for i in expnum_list]))].groupby(["camera"])

for camera in fiberflat_analogs.groups:
    fiberflats = fiberflat_analogs.get_group(camera).sort_values("exptime")
    print(fiberflats.to_string())
    
    # let's pre-process the all fiberflat frames
    cfiberflat_paths = []
    for _, fiberflat_frame in fiberflats.iterrows():
        fiberflat_path = fiberflat_frame["path"]
        fiberflat_expnum = fiberflat_frame["expnum"]

        cfiberflat_paths.append(out_main_path.format(kind="cfiberflat", camera=camera, expnum=fiberflat_expnum))

    imageMethod.createMasterFrame_drp(
        in_images=",".join(cfiberflat_paths),
        out_image=out_calib_path.format(mjd="super", kind="mfiberflat", camera=camera, exptime=int(fiberflats.exptime.min())),
    )

    imagetyp spec    mjd camera    expnum  exptime  argon   neon  ldls   hgne  xenon  krypton                                                                                                                      path
976   object  sp1  60041     b1  00000694     30.0  False  False  True  False  False    False  /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/sas/sdsswork/data/lvm/lco/60041/sdR-s-b1-00000694.fits.gz
914   object  sp1  60041     b1  00000695     90.0  False  False  True  False  False    False  /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/sas/sdsswork/data/lvm/lco/60041/sdR-s-b1-00000695.fits.gz
[INFO]: input frames: ./data/lvm-cfiberflat-b1-00000694.fits,./data/lvm-cfiberflat-b1-00000695.fits
[WARNING]: not all exptime = 30.0. Scaling images
[INFO]: combining 2 frames into master frame
[INFO]: updating header for new master frame './data/lvm-super-mfiberflat-b1-30.fits'
     imagetyp spec    mjd camera    expnum  exptime  argon   neon  ldls